# Modern Data Science
**(Module 07: Natural Language Processing)**

---
- Materials in this module include resources collected from various open-source online repositories.
- You are free to use, change and distribute this package.
- If you found any issue/bug for this document, please submit an issue at [tulip-lab/mds](https://github.com/tulip-lab/mds/issues)

Prepared by and for 
**Student Members** |
2006-2019 [TULIP Lab](http://www.tulip.org.au)

---

# Session H - Topic Model - LDA

In this notebook, we use unsupervised learning models to cluster unlabeled documents into different groups, visualize the results and identify their latent topics/structures.

## Contents

<ul>
<li>[Part 1: Load Data](#Part-1:-Load-Data)
<li>[Part 2: Tokenizing and Stemming](#Part-2:-Tokenizing-and-Stemming)
<li>[Part 3: TF-IDF](#Part-3:-TF-IDF)
<li>[Part 4: K-means clustering](#Part-4:-K-means-clustering)
<li>[Part 5: Topic Modeling - Latent Dirichlet Allocation](#Part-5:-Topic-Modeling---Latent-Dirichlet-Allocation)
</ul>

# Part 1: Load Data

In [ ]:
# !pip install sklearn
# !pip install lda

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import nltk
import re
import os

from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import lda

Read data from files. In summary, we have 100 titles and 100 synoposes (combined from imdb and wiki).

In [ ]:
# #import three lists: titles and wikipedia synopses
# titles = open('title_list.txt').read().split('\n')
# titles = titles[:100] #ensures that only the first 100 are read in

# #The wiki synopses and imdb synopses of each movie is seperated by the keywords "BREAKS HERE". 
# #Each synoposes may consist of multiple paragraphs.
# synopses_wiki = open('synopses_list_wiki.txt').read().split('\n BREAKS HERE')
# synopses_wiki = synopses_wiki[:100]

# synopses_imdb = open('synopses_list_imdb.txt').read().split('\n BREAKS HERE')
# synopses_imdb = synopses_imdb[:100]

In [ ]:
import wget

In [ ]:
# import the datasets
title = "https://github.com/hudrizzle/Movie-Topics-Clustering/raw/master/title_list.txt"
titles = wget.download(titles)

synopses_wiki = "https://github.com/hudrizzle/Movie-Topics-Clustering/raw/master/synopses_list_wiki.txt"
synopses_wiki = wget.download(synopses_wiki)

synopses_imdb = "https://github.com/hudrizzle/Movie-Topics-Clustering/raw/master/synopses_list_imdb.txt"
synopses_imdb = wget.download(synopses_imdb)

In [ ]:
#import three lists: titles and wikipedia synopses
titles = open(DataSet).read().split('\n')
titles = titles[:100]  #ensures that only the first 100 are read in

synopses_wiki = open(synopses_wiki).read().split('\n BREAKS HERE')
synopses_wiki = synopses_wiki[:100]

synopses_imdb = open(synopses_imdb).read().split('\n BREAKS HERE')
synopses_imdb = synopses_imdb[:100]

In [ ]:
#Combine imdb and wiki to get full synoposes for the top 100 movies. 
synopses = []
for i in range(len(synopses_wiki)):
    item = synopses_wiki[i] + synopses_imdb[i]
    synopses.append(item)
    
#Because these synopses have already been ordered in popularity order, 
#we just need to generate a list of ordered numbers for future usage.
ranks = range(len(titles))

# Part 2: Tokenizing and Stemming

Load stopwords and stemmer function from NLTK library.
Stop words are words like `a`, `the`, or `in` which don't convey significant meaning.
Stemming is the process of breaking a word down into its root.

In [ ]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english')

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

In [ ]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

def tokenization_and_stemming(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
#     tokens=[]
#     for sent in nltk.sent_tokenize(text):
#         for word in nltk.word_tokenize(sent):
#             if word not in stopwords:
#                 tokens.append(word);   
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

def tokenization(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [ ]:
tokenization_and_stemming("she looked at her father's arm.")

Use our defined functions to analyze (i.e. tokenize, stem) our synoposes.

In [ ]:
docs_stemmed = []
docs_tokenized = []
for i in synopses:
    tokenized_and_stemmed_results = tokenization_and_stemming(i)
    docs_stemmed.extend(tokenized_and_stemmed_results)
    
    tokenized_results = tokenization(i)
    docs_tokenized.extend(tokenized_results)

Create a mapping from stemmed words to original tokenized words for result interpretation.

In [ ]:
vocab_frame_dict = {docs_stemmed[x]:docs_tokenized[x] for x in range(len(docs_stemmed))}
print (vocab_frame_dict['angel'])

# Part 3: TF-IDF

In [ ]:
#define vectorizer parameters
tfidf_model = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(synopses) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " synoposes and " + str(tfidf_matrix.shape[1]) + " terms.")

In [ ]:
tfidf_model.get_params()

Save the terms identified by TF-IDF.

In [ ]:
tf_selected_words = tfidf_model.get_feature_names()

### Calculate Document Similarity

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
cos_matrix = cosine_similarity(tfidf_matrix)
print (cos_matrix)

# Part 4: K-means clustering for Topic

In [ ]:
from sklearn.cluster import KMeans

num_clusters = 3
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

## 4.1. Analyze K-means Result

In [ ]:
# create DataFrame films from all of the input files.
films = { 'title': titles, 'rank': ranks, 'cluster': clusters}
frame = pd.DataFrame(films, columns = ['rank', 'title', 'cluster'])
frame.head(10)

In [ ]:
print ("Number of films included in each cluster:")
frame['cluster'].value_counts().to_frame()

In [ ]:
grouped = frame['rank'].groupby(frame['cluster'])
print ("Average rank (1 to 100) per cluster:")
grouped.mean().to_frame()

In [ ]:

print ("<Document clustering result by K-means>")

# km.cluster_centers_ denotes the importances of each items in centroid.
# need to sort it in descending order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:" ,)
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: # get the top 6 words of each cluster
        Cluster_keywords_summary[i].append(vocab_frame_dict[tf_selected_words[ind]])
        print (vocab_frame_dict[tf_selected_words[ind]] + ", ",)
    print()
    cluster_movies = frame.loc[frame.cluster == i, 'title'].values.tolist()
    print ("Cluster " + str(i) + " titles (" + str(len(cluster_movies)) + " movies): " )
    print (", ".join(cluster_movies), '\n')

## 4.2. Plot K-means Result

In [ ]:
pca = decomposition.PCA(n_components=6)
tfidf_matrix_np=tfidf_matrix.toarray()
pca.fit(tfidf_matrix_np)
X = pca.transform(tfidf_matrix_np)

xs, ys = X[:, 0], X[:, 1]

#set up colors per clusters using a dict
cluster_colors = {0: 'g', 1: 'b', 2: 'r', 3: 'y', 4:'k',5:'m'}
#set up cluster names using a dict
cluster_names = {}
for i in range(num_clusters):
    cluster_names[i] = ", ".join(Cluster_keywords_summary[i])

In [ ]:
%matplotlib inline 

#create data frame with PCA cluster results
df = pd.DataFrame(dict(x=xs, y=ys, label=clusters, title=titles)) 
groups = df.groupby(clusters)

# set up plot
fig, ax = plt.subplots(figsize=(10, 6))
#Set color for each cluster/group
for name, group in groups:
    ax.plot(group.x, group.y, marker='o', linestyle='', ms=12, 
            label=cluster_names[name], color=cluster_colors[name], 
            mec='none')

ax.legend(numpoints=1,loc=4)  #show legend with only 1 point, position is right bottom.

plt.show() #show the plot

# Part 5: Topic Modeling - Latent Dirichlet Allocation

In [ ]:
#Now we use synopses to build a 100*551 matrix (terms)
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=3, learning_method = 'online')

tfidf_matrix_lda = (tfidf_matrix * 100)
tfidf_matrix_lda = tfidf_matrix_lda.astype(int)

In [ ]:
lda.fit(tfidf_matrix_lda)

<li> "model.topic_word_" saves the importance of tf_selected_words in LDA model, i.e. words similarity matrix
<li> The shape of "model.topic_word_" is (n_topics,num_of_selected_words)
<li> "model.doc_topic_" saves the document topic results, i.e. document topic matrix.
<li> The shape of "model.doc_topic_" is (num_of_documents, n_topics)

In [ ]:
topic_word = lda.components_
print (topic_word.shape)

In [ ]:
n_top_words = 7
topic_keywords_list = []
for i, topic_dist in enumerate(topic_word):
    #Here we select top(n_top_words-1)
    lda_topic_words = np.array(tf_selected_words)[np.argsort(topic_dist)][:-n_top_words:-1] 
    for j in range(len(lda_topic_words)):
        lda_topic_words[j] = vocab_frame_dict[lda_topic_words[j]]
    topic_keywords_list.append(lda_topic_words.tolist())

In [ ]:
topic_keywords_list

In [ ]:
doc_topic = lda.transform(tfidf_matrix_lda)
print (doc_topic.shape)

In [ ]:
topic_doc_dict = {}
print ("<Document clustering result by LDA>")
for i in range(len(doc_topic)):
    topicID = doc_topic[i].argmax()
    if topicID not in topic_doc_dict:
        topic_doc_dict[topicID] = [titles[i]]
    else:
        topic_doc_dict[topicID].append(titles[i])
for i in topic_doc_dict:
    print ("Topic " + str(i) + " words: " + ", ".join(topic_keywords_list[i]))
    print ("Topic " + str(i) + " titles (" + str(len(topic_doc_dict[i])) + " movies): " )
    print (', '.join(topic_doc_dict[i]))
    print ()

We got a similar set of three clusters/topics as those we got with KMeans, but the topic keywords are different.

